In [18]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import time
from scipy.optimize import linprog
from cvxopt import matrix, solvers
from scipy.sparse import issparse
from osqp import OSQP
from scipy.linalg import qr
import matplotlib
matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt

### question 4

In [22]:
def load():
    data = loadmat('/Users/annikalaw/Downloads/QP_Test.mat')
    
    # regularize some of the matrices
    H = np.asarray(data['H'], dtype=np.float64)
    g = np.asarray(data['g'], dtype=np.float64).ravel()
    C = np.asarray(data['C'], dtype=np.float64)
    l = np.asarray(data['l'], dtype=np.float64).ravel()
    u = np.asarray(data['u'], dtype=np.float64).ravel()
    
    H = 0.5*(H + H.T) + 1e-3*np.eye(H.shape[0])  # to ensure positive definiteness
    g = g * 0.1  # control penalty adjustment
    
    return H, g, C, l, u

def enforce_full_rank(A, tol=1e-6):
    Q, R, P = qr(A.T, pivoting=True) # to ensure matrix has full row rank
    rank = np.sum(np.abs(np.diag(R)) > tol * np.abs(R[0,0]))
    return A[P[:rank], :]

def solve():
    H, g, C, l, u = load()
    n = len(g)
    
    # bound constraints (l ≤ x ≤ u)
    G_bound = np.vstack([-np.eye(n), np.eye(n)])
    h_bound = np.hstack([-l, u])
    
    # rate constraints (Δu_min ≤ Δu ≤ Δu_max)
    Δu_max, Δu_min = 10, -10 
    D = np.eye(n) - np.eye(n, k=-1)
    D = D[1:]  # n-1 × n difference matrix
    
    # combining constraints
    G_full = np.vstack([G_bound, -D, D])
    h_full = np.hstack([h_bound, Δu_min*np.ones(n-1), Δu_max*np.ones(n-1)])
    
    # remove the dependent constraints
    G_reduced = enforce_full_rank(G_full)
    h_reduced = h_full[:G_reduced.shape[0]]
    
    P = matrix(H)
    q = matrix(g)
    G = matrix(G_reduced)
    h = matrix(h_reduced)
    
    solvers.options.update({
        'show_progress': True,
        'maxiters': 1000,
        'reltol': 1e-6,
        'feastol': 1e-6,
        'kktreg': 1e-6,
        'refinement': 2
    })
    
    try:
        start_time = time.time() # start time
        solution = solvers.qp(P, q, G, h)
        end_time = time.time() # end time
        u_opt = np.array(solution['x']).flatten()
        
        cpu_time = end_time - start_time
        iterations = solution['iterations']
        opt_value = solution['primal objective']
        
        # stats
        print("\n---- Solution ----")
        print(f"{'CPU Time:':<20} {cpu_time:.4f} seconds")
        print(f"{'Iterations:':<20} {iterations}")
        print(f"{'Optimal Value:':<20} {opt_value:.6e}")
        
        # plot visualization
        plt.figure(figsize=(12, 6))
        plt.plot(u_opt[::2], 'r-', linewidth=2, label='Pump 1')
        plt.plot(u_opt[1::2], 'b--', linewidth=2, label='Pump 2')
        plt.xlabel('Time step', fontsize=12)
        plt.ylabel('Control value', fontsize=12)
        plt.title('4-Tank Optimal Control', fontsize=14)
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        return u_opt, cpu_time, iterations, opt_value
        
    except Exception as e:
        return None, None, None, None
    
if __name__ == '__main__':
    controls, cpu_time, iterations, opt_value = solve()

     pcost       dcost       gap    pres   dres
 0:  2.6663e+05 -6.4258e+05  1e+06  3e-01  8e+01
 1:  3.5909e+04 -2.8122e+05  3e+05  7e-02  2e+01
 2:  9.4519e+03 -3.3094e+05  4e+05  5e-02  2e+01
 3: -3.8719e+02 -2.1968e+04  2e+04  3e-03  1e+00
 4: -4.7779e+02 -1.3086e+03  8e+02  1e-04  3e-02
 5: -4.8103e+02 -4.9112e+02  1e+01  1e-06  4e-04
 6: -4.8103e+02 -4.8113e+02  1e-01  1e-08  4e-06
 7: -4.8103e+02 -4.8104e+02  1e-03  1e-10  4e-08
 8: -4.8103e+02 -4.8103e+02  1e-05  1e-12  4e-10
Optimal solution found.

---- Solution ----
CPU Time:            0.2737 seconds
Iterations:          8
Optimal Value:       -4.810341e+02


### question 6

In [26]:
#q6p1

class primal_QP:
    def __init__(self, H, g, C=None, dl=None, du=None, l=None, u=None):
        self.H = H
        self.g = g
        self.C = C
        self.dl = dl
        self.du = du
        self.l = l
        self.u = u
        
        # dimensions
        self.n = H.shape[0]
        self.m = C.shape[1] if C is not None else 0
        self.solution = None
    
    def find_point(self, tol=1e-6):
        n = self.n
        
        # bounds
        if self.C is None:
            if self.l is not None and self.u is not None:
                x = 0.5 * (self.l + self.u)
                if self.is_feasible(x, tol):
                    return x
            return np.zeros(n)  # fall back
        
        c = np.zeros(n + 1)
        c[-1] = 1  # minimizing slack
        
        # inequality constraints
        A_ub = []
        b_ub = []
        
        # linear constraints
        if self.C is not None:
            C_T = self.C.T
            if self.du is not None:
                A_ub.append(np.hstack([C_T, -np.ones((self.m, 1))]))
                b_ub.append(self.du)
            if self.dl is not None:
                A_ub.append(np.hstack([-C_T, -np.ones((self.m, 1))]))
                b_ub.append(-self.dl)
        
        # bound constraints
        if self.l is not None:
            A_ub.append(np.hstack([-np.eye(n), -np.ones((n, 1))]))
            b_ub.append(-self.l)
        if self.u is not None:
            A_ub.append(np.hstack([np.eye(n), -np.ones((n, 1))]))
            b_ub.append(self.u)
        
        if A_ub:
            A_ub = np.vstack(A_ub)
            b_ub = np.hstack(b_ub)
            bounds = [(None, None)] * n + [(0, None)]
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
            
            if res.success and res.x[-1] < tol:
                return res.x[:-1]
        
        x = np.zeros(n)
        if self.is_feasible(x, tol):
            return x
        
        raise ValueError("no feasible point found")
    
    def is_feasible(self, x, tol=1e-6): # cehck for feasibility
        if self.l is not None and np.any(x < self.l - tol):
            return False
        if self.u is not None and np.any(x > self.u + tol):
            return False
        if self.C is not None:
            Cx = self.C.T @ x
            if self.dl is not None and np.any(Cx < self.dl - tol):
                return False
            if self.du is not None and np.any(Cx > self.du + tol):
                return False
        return True
    
    def equality_qp(self, H, g, A_eq):
        n = H.shape[0]
        k = A_eq.shape[0] if A_eq.size > 0 else 0
        
        if k == 0:
            try:
                return np.linalg.solve(H, -g), np.array([])
            except np.linalg.LinAlgError:
                return np.linalg.lstsq(H, -g, rcond=None)[0], np.array([])
        
        KKT = np.block([
            [H, A_eq.T],
            [A_eq, np.zeros((k, k))]
        ])
        rhs = np.hstack([-g, np.zeros(k)])
        
        try:
            sol = np.linalg.solve(KKT, rhs)
        except np.linalg.LinAlgError:
            sol = np.linalg.lstsq(KKT, rhs, rcond=None)[0]
        
        return sol[:n], sol[n:n+k]
    
    def solve(self, tol=1e-6, max_iter=1000, verbose=True):
        start_time = time.time()
        x = self.find_point(tol) # find feasible point
        
        # combine constraints
        A_all, b_all, constraint_types = [], [], []
        
        # bound constraints
        if self.l is not None:
            A_all.append(np.eye(self.n))
            b_all.append(self.l)
            constraint_types.extend(['lower_bound']*self.n)
        
        if self.u is not None:
            A_all.append(-np.eye(self.n))
            b_all.append(-self.u)
            constraint_types.extend(['upper_bound']*self.n)
        
        if self.C is not None: # linear constraints
            C_T = self.C.T
            if self.dl is not None:
                A_all.append(-C_T)
                b_all.append(-self.dl)
                constraint_types.extend(['linear_lower']*self.m)
            if self.du is not None:
                A_all.append(C_T)
                b_all.append(self.du)
                constraint_types.extend(['linear_upper']*self.m)
        
        if not A_all:  # unconstrained bounds
            p, _ = self.equality_qp(self.H, self.g, np.zeros((0, self.n)))
            self.solution = {
                'x': p, 'lambda_bounds': None, 'lambda_ineq': None,
                'iterations': 1, 'cpu_time': time.time()-start_time,
                'success': True, 'active_set': []
            }
            return self.solution
        
        A_all = np.vstack(A_all)
        b_all = np.hstack(b_all)
        
        # active set
        active_set = []
        residuals = A_all @ x - b_all
        for i in range(len(residuals)):
            if abs(residuals[i]) < tol:
                active_set.append(i)
        
        iterations = 0
        converged = False
        
        while not converged and iterations < max_iter:
            iterations += 1
            
            # equality QP
            A_eq = A_all[active_set] if active_set else np.zeros((0, self.n))
            p, lambda_active = self.equality_qp(self.H, self.H@x + self.g, A_eq)
            
            # convergence
            if np.linalg.norm(p) < tol:
                if not active_set or np.all(lambda_active >= -tol):
                    converged = True
                else:
                    idx = active_set[np.argmin(lambda_active)]
                    active_set.remove(idx) # remove negative multiplier
                    if verbose: continue
            
            alpha = 1.0 # step length
            
            for i in range(len(A_all)):
                if i not in active_set:
                    a_i = A_all[i]
                    b_i = b_all[i]
                    denom = a_i @ p
                    if denom > tol:
                        alpha_i = (b_i - a_i @ x) / denom
                        if 0 < alpha_i < alpha:
                            alpha = alpha_i
            
            x = x + alpha * p
        
        lambda_all = np.zeros(len(A_all))
        for idx, i in enumerate(active_set):
            lambda_all[i] = lambda_active[idx]
        
        # dual variables
        lambda_bounds = np.zeros(2*self.n)
        lambda_ineq = np.zeros(2*self.m) if self.m > 0 else None
        
        for i, typ in zip(range(len(A_all)), constraint_types):
            if typ == 'lower_bound':
                lambda_bounds[i] = lambda_all[i]
            elif typ == 'upper_bound':
                lambda_bounds[self.n + i - self.n] = lambda_all[i]
            elif typ == 'linear_lower' and lambda_ineq is not None:
                lambda_ineq[i - 2*self.n] = lambda_all[i]
            elif typ == 'linear_upper' and lambda_ineq is not None:
                lambda_ineq[self.m + i - 2*self.n - self.m] = lambda_all[i]
        
        self.solution = {
            'x': x,
            'lambda_bounds': lambda_bounds,
            'lambda_ineq': lambda_ineq,
            'iterations': iterations,
            'cpu_time': time.time() - start_time,
            'success': converged,
            'active_set': active_set
        }
        
        if verbose: # stats
            print("\n---- Solution ----")
            print(f"Iterations: {iterations}")
            print(f"CPU time: {self.solution['cpu_time']:.4f} sec")
            print(f"Optimal value: {0.5*x.T@self.H@x + self.g.T@x:.4f}")
        
        return self.solution

In [27]:
#q6p2
data = loadmat('/Users/annikalaw/Downloads/QP_Test.mat')
H = data['H']
g = data['g'].flatten()
C = data['C']
d_l = data['dl'].flatten()
d_u = data['du'].flatten()
l = data['l'].flatten()
u = data['u'].flatten()

qp = primal_QP(H, g, C, d_l, d_u, l, u)
solution = qp.solve(verbose=True)

# reshape solution
N = len(solution['x']) // 2
U_opt = solution['x'].reshape(N, 2)
time_steps = np.arange(N)  # time axis

# visualisation
plt.figure(figsize=(10, 5))
plt.plot(time_steps, U_opt[:, 0], 'b-', linewidth=2, label='Input 1 (u₁)')
plt.plot(time_steps, U_opt[:, 1], 'r--', linewidth=2, label='Input 2 (u₂)')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Control Input Value', fontsize=12)
plt.title('Optimal Control Inputs for 4-Tank System', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.savefig('qp_sol_q6.png', dpi=300, bbox_inches='tight')
print("saved as 'qp_sol_q6.png'")



---- Solution ----
Iterations: 46
CPU time: 0.1526 sec
Optimal value: -49359.8328
saved as 'qp_sol_q6.png'


### question 8

In [28]:
def load():
    data = loadmat('/Users/annikalaw/Downloads/QP_Test.mat')
    
    # convert to numpy arrays
    def convert(var, default_name):
        arr = data.get(var, data.get(default_name, None))
        if issparse(arr): # sparse matrices
            arr = arr.toarray()
        arr = np.array(arr, dtype=np.float64)
        return arr

    H = convert('H', 'H').T  # cvxopt compatibility
    g = convert('g', 'g').reshape(-1) 
    C = convert('C', 'C').T 
    d_l = convert('d_l', 'dl').reshape(-1)  
    d_u = convert('d_u', 'du').reshape(-1) 
    l = convert('l', 'l').reshape(-1)  
    u = convert('u', 'u').reshape(-1)  
    
    return H, g, C, d_l, d_u, l, u

def create(H, g, C, d_l, d_u, l, u):
    def make_cvxopt_matrix(arr):
        arr = np.ascontiguousarray(arr, dtype=np.float64)
        return matrix(arr, tc='d')
    P = make_cvxopt_matrix(H)
    q = make_cvxopt_matrix(g) 
    
    # inequality constraints: (l <= x <= u) and (d_l <= C^T x <= d_u)
    n = len(g)
    G = make_cvxopt_matrix(np.vstack([
        -np.eye(n),  # -x <= -l
        np.eye(n),   # x <= u
        C.T,         # C^T x <= d_u
        -C.T         # -C^T x <= -d_l
    ])) 
    h = make_cvxopt_matrix(np.hstack([
        -l,         
        u,           
        d_u,         
        -d_l         
    ])) 

    A,b = None,None
    return P, q, G, h, A, b

def solve_qp():
    start_time = time.time()
    try:
        H, g, C, d_l, d_u, l, u = load()
    except Exception as e: return None
    
    try:
        P, q, G, h, A, b = create(H, g, C, d_l, d_u, l, u)
    except Exception as e: return None
    
    solvers.options.update({ # solver options
        'show_progress': True,
        'maxiters': 100,
        'reltol': 1e-6,
        'abstol': 1e-6,
        'feastol': 1e-6
    })

    try:
        solution = solvers.qp(P, q, G, h, A, b)
    except Exception as e: return None
    cpu_time = time.time() - start_time # calculate time

    # results
    x_opt = np.array(solution['x'])
    primal_obj = solution['primal objective']
    iterations = solution['iterations']
    status = solution['status']
    
    # visualisation
    plt.figure(figsize=(10, 5))
    plt.plot(x_opt[::2], 'r-', label='Control Input 1')
    plt.plot(x_opt[1::2], 'b--', label='Control Input 2')
    plt.xlabel('Time Step')
    plt.ylabel('Control Value')
    plt.title('Optimal Control Inputs')
    plt.legend()
    plt.grid(True)
    plt.savefig('qp_sol_q8.png')
    print("saved as 'qp_sol_q8.png'")
    plt.show()
    
    # stats
    print("\n---- Solution ----")
    print(f"Iterations: {iterations}")
    print(f"CPU Time: {cpu_time:.4f} seconds")
    print(f"Optimal value: {primal_obj:.6f}")

    
    return {
        'x': x_opt,
        'objective': primal_obj,
        'iterations': iterations,
        'status': status
    }

if __name__ == '__main__':
    solution = solve_qp()

     pcost       dcost       gap    pres   dres
 0:  3.5024e+04 -8.2180e+06  1e+07  8e-02  2e-14
 1:  5.3495e+03 -9.6478e+05  1e+06  8e-03  8e-15
 2: -3.5752e+04 -1.5177e+05  1e+05  7e-04  1e-15
 3: -4.5159e+04 -8.7532e+04  4e+04  2e-04  9e-16
 4: -4.7942e+04 -6.3582e+04  2e+04  2e-16  1e-15
 5: -4.8463e+04 -4.9026e+04  6e+02  2e-16  1e-15
 6: -4.8528e+04 -4.8555e+04  3e+01  2e-16  1e-15
 7: -4.8536e+04 -4.8538e+04  2e+00  2e-16  9e-16
 8: -4.8537e+04 -4.8537e+04  2e-01  2e-16  1e-15
 9: -4.8537e+04 -4.8537e+04  2e-02  2e-16  1e-15
Optimal solution found.
saved as 'qp_sol_q8.png'

---- Solution ----
Iterations: 9
CPU Time: 0.1310 seconds
Optimal value: -48536.561666
